Projekt końcowy

Lending Club to firma pożyczkowa typu peer-to-peer, która łączy pożyczkobiorców z inwestorami za pośrednictwem platformy internetowej. Obsługuje osoby, które potrzebują pożyczek osobistych w wysokości od 1000 do 40 000 USD. Pożyczkobiorcy otrzymują pełną kwotę udzielonej pożyczki pomniejszoną o opłatę początkową, która jest uiszczana firmie. Inwestorzy kupują weksle zabezpieczone osobistymi pożyczkami i płacą Lending Club opłatę za usługę. Firma Lending Club udostępnia dane o wszystkich pożyczkach udzielonych za pośrednictwem swojej platformy w określonych okresach.
Na potrzeby tego projektu zostały użyte dane dotyczące pożyczek udzielonych za pośrednictwem Lending Club na przestrzeni lat 2007 -2011. Każda pożyczka jest opatrzona informacją o tym, czy ostatecznie została spłacona (Fully Paid lub Charged off w kolumnie loan_status). Twoim zadaniem jest zbudowanie modelu klasyfikacyjnego, który na podstawie tych danych będzie przewidywał z określoną dokładnością, czy potencjalny pożyczkobiorca spłaci swój dług z tytułu zaciągniętej pozyczki. Do zbioru danych dołączony jest plik z opisem wszystkich zmiennych oraz plik „FICO Score ranged.pdf”, w którym dokładnie opisano znaczenie jednej z kolumn.

Poniżej zaprezentowane są poszczególne etapy analizy, których wykonanie jest konieczne do zaliczenia projektu oraz ich punktacja:

- Obróbka danych (Data Processing) (70pkt) – jako doświadczony Data Scientist zapewne znasz poszczególne kroki, które należy wykonać na tym etapie, więc nie będziemy ich tutaj wyszczególniać.
- EDA, czyli obszerna eksploracja danych (100pkt) Opisz wnioski płynące z każdego wykresu, swoje hipotezy poprzyj testami statystycznymi takimi jak np. t-test lub Chi-square. Dodatkowo odpowiedz na poniższe pytania:
    - W jaki sposób wynik FICO wiąże się z prawdopodobieństwem spłacenia pożyczki przez pożyczkobiorcę?
    - W jaki sposób wiek kredytowy wiąże się z prawdopodobieństwem niewykonania zobowiązania i czy ryzyko to jest niezależne lub związane z wynikiem FICO
    - W jaki sposób status kredytu hipotecznego na dom wiąże się z prawdopodobieństwem niewypłacalności?
    - W jaki sposób roczny dochód wiąże się z prawdopodobieństwem niewykonania zobowiązania?
    - W jaki sposób historia zatrudnienia wiąże się z prawdopodobieństwem niewykonania zobowiązania?
    - Jak wielkość żądanej pożyczki jest powiązana z prawdopodobieństwem niewykonania zobowiązania?
- Feature Engineering – utwórz 20 nowych zmiennych (60 pkt)
- Modelowanie (150 pkt)
- Wykonaj klasteryzację danych (wypróbuj do tego celu kilka metod, min. 3) i sprawdź, czy występują jakieś segmenty pożyczkobiorców, wykorzystaj odpowiednie metody do określenia optymalnej liczby klastrów (40 pkt)
- Wytrenuj 5 różnych modeli, wykorzystując do każdego inny algorytm, a następnie porównaj ich działanie, za metrykę oceny jakości modelu przyjmij AUROC score. (50pkt)
- Sprawdź działanie wcześniej użytych metod na skompresowanych danych za pomocą PCA, porównaj wyniki (AUROC score) z modelami wytrenowanymi w poprzednim podpunkcie. (20pkt)
- Zbuduj finalny model, którego AUROC score będzie >= 80%, pamiętaj o doborze istotnych zmiennych, kroswalidacji oraz dostrojeniu parametrów modelu, pomyśl również o zbalansowaniu klas. (40pkt)

Za całość do zdobycia jest 380 punktów. Do zaliczenia projektu potrzeba minimum 300 pkt.
Powodzenia!

# Rozwiązanie

### Przygotowanie danych

Na początek importuję potrzebne biblioteki. W trakcie pracy nad projektem, będę sukcesywnie dodawać kolejne potrzebne biblioteki.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Ze względu na wielkość pliku, zmieniam wyświetlanie liczby kolumn oraz znaków w wierszu po to, żeby wyświetlały się wszystkie.

In [4]:
pd.set_option('max_columns', None)
pd.set_option("max_colwidth", None)

Wczytuję oba pliki. Plik z danymi do analizy nazywam "df", a z opisem kolumn "legend".

In [5]:
df = pd.read_csv("Loan_data.csv", delimiter=",", low_memory=False)
legend = pd.read_csv("LCDataDictionary.csv", delimiter=",")

Sprawdzam, czy zostały wczytane prawidłowo i jednocześnie zerkam, co się w nich znajduje.

In [6]:
legend.head()

,LoanStatNew,Description,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,acc_now_delinq,The number of accounts on which the borrower is now delinquent.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,acc_open_past_24mths,Number of trades opened in past 24 months.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,addr_state,The state provided by the borrower in the loan application,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,all_util,Balance to credit limit on all trades,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,annual_inc,The self-reported annual income provided by the borrower during registration.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
print(df.shape)
df.head()

(42536, 151)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1077501,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=1077501,Borrower added on 12/22/11 > I need to upgrade my business technologies.<br>,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,735.0,739.0,1.0,NaN,NaN,3.0,0.0,13648.0,83.7%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,Jan-2015,171.62,NaN,Aug-2018,739.0,735.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://lendingclub.com/browse/loanDetail.action?loan_id=1077430,Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike. I only need this money because the deal im looking at is to good to pass up.<br><br> Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike.I only need this money because the deal im looking at is to good to pass up. I have finished college 

Zaczynam od wstępnego czyszczenia danych z niepotrzebnych kolumn. Na początek usuwam wszystkie kolumny bez danych oraz te, w których jest za dużo NaNów.

In [8]:
# na początek sprawdzam, jak dużo jest NaN w każdej kolumnie
nan_list = df.isna().sum().tolist()
print(nan_list)

[0, 42536, 1, 1, 1, 1, 1, 1, 1, 1, 2627, 1113, 1, 5, 1, 1, 1, 1, 1, 13294, 1, 14, 1, 1, 1, 30, 30, 1, 1, 30, 26927, 38885, 30, 30, 1, 91, 30, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 84, 1, 39787, 5, 1, 1, 146, 42536, 1, 1, 42536, 42536, 42536, 30, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 146, 30, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 1366, 106, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 1, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 42536, 1, 1, 42376, 42376, 42376, 42376, 42376, 42376]


In [9]:
# usuwam te kolumny, w których są tylko NaN
df = df.dropna(axis=1, how="all")

In [10]:
# ponownie sprawdzam, jak dużo jest NaN w każdej kolumnie
nan_list = df.isna().sum().tolist()
print(nan_list)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 2627, 1113, 1, 5, 1, 1, 1, 1, 1, 13294, 1, 14, 1, 1, 1, 30, 30, 1, 1, 30, 26927, 38885, 30, 30, 1, 91, 30, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 84, 1, 39787, 5, 1, 1, 146, 1, 1, 30, 146, 30, 1366, 106, 1, 1, 1, 42376, 42376, 42376, 42376, 42376, 42376]


Widzę 2 rzeczy:
- po pierwsze w niemal każdej kolumnie poza ID znajdują się jakieś NaN, co wskazuje, że może być jakiś rząd, który mam puste wartości
- po drugie nadal znajdują się kolumny, które są w dużej mierze puste

In [11]:
# usuwam kolumny z dużą liczba pustych pól (więcej niż 20%)
df = df.dropna(axis=1, thresh=0.8*len(df))

In [12]:
# ponownie sprawdzam, jak dużo jest NaN w każdej kolumnie
nan_list = df.isna().sum().tolist()
print(nan_list)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 2627, 1113, 1, 5, 1, 1, 1, 1, 1, 1, 14, 1, 1, 1, 30, 30, 1, 1, 30, 30, 30, 1, 91, 30, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 84, 1, 5, 1, 1, 146, 1, 1, 30, 146, 30, 1366, 106, 1, 1, 1]


In [13]:
# usuwam wiersze, które zawierają więcej niż 10 pustych pól 
df = df.dropna(axis=0, thresh=10)

In [14]:
# ponownie sprawdzam, jak dużo jest NaN w każdej kolumnie
nan_list = df.isna().sum().tolist()
print(nan_list)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 2626, 1112, 0, 4, 0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 29, 29, 0, 0, 29, 29, 29, 0, 90, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 83, 0, 4, 0, 0, 145, 0, 0, 29, 145, 29, 1365, 105, 0, 0, 0]


In [15]:
# sprawdzam, jak wygląda aktualnie zbiór danych i widzę, że nadal składa się z 59 kolumn - czyli nadal za dużo do ML
print(df.shape)

(42535, 59)


Przechodzę do dalszej analizy ilościowej danych - dopiero następnie zajmę się analizą jakościową.
Na początek zamierzam usunąć dwa rodzaje kolumn: takie w których żadna wartość się nie powtarza oraz takie które zawierają tylko jedną wartość. Żadna z tych kolumn nie będzie przydatna przy uczeniu maszynowym.

In [18]:
# sprawdzam, ile unikalnych wartości znajduje się w każdej z kolumn
df.nunique()

loan_amnt                    898
funded_amnt                 1051
funded_amnt_inv             9248
term                           2
int_rate                     394
installment                16459
grade                          7
sub_grade                     35
emp_title                  30658
emp_length                    11
home_ownership                 5
annual_inc                  5597
verification_status            3
issue_d                       55
loan_status                    4
purpose                       14
title                      21264
zip_code                     837
addr_state                    50
dti                         2894
delinq_2yrs                   12
earliest_cr_line             530
fico_range_low                44
fico_range_high               44
inq_last_6mths                28
open_acc                      44
pub_rec                        6
revol_bal                  22709
revol_util                  1119
total_acc                     83
total_pymn

In [19]:
# widzę, że w kolumnach 'id' i 'url' żadne wartości się nie powtarzają, a kilka kolumn ma tylko 1 wartość, więc je usuwam
unique_values = df.nunique()
to_drop = unique_values[unique_values == 1].index
df = df.drop(to_drop, axis=1)
to_drop = unique_values[unique_values == 42535].index
df = df.drop(to_drop, axis=1)
# sprawdzam wielkość bazy danych oraz jakie kolumny zostały
print(df.shape)
print(unique_values)

(42535, 47)
loan_amnt                    898
funded_amnt                 1051
funded_amnt_inv             9248
term                           2
int_rate                     394
installment                16459
grade                          7
sub_grade                     35
emp_title                  30658
emp_length                    11
home_ownership                 5
annual_inc                  5597
verification_status            3
issue_d                       55
loan_status                    4
purpose                       14
title                      21264
zip_code                     837
addr_state                    50
dti                         2894
delinq_2yrs                   12
earliest_cr_line             530
fico_range_low                44
fico_range_high               44
inq_last_6mths                28
open_acc                      44
pub_rec                        6
revol_bal                  22709
revol_util                  1119
total_acc                     8

Kolejnym krokiem jest analiza jakościowa - co oznacza, że muszę ustalić, które z kolumn zawierają dane z przyszłości (czyli dane, do których nie ma dostępu przy podejmowaniu decyzji kredytowej). W tym celu tworzę tabelę "preview" z opisem kolumn, które pozostały w "df".

In [20]:
# w "legend" zmieniam nazwę pierwszej kolumny na "name" i wielkość liter w nazwie drugiej kolumny dla ułatwienia
# usuwam też wszystkie kolejne kolumny, które nie zawierają żadnych danych
legend = legend.rename(columns={"LoanStatNew":"name", "Description":"description"}).dropna(axis=1, how="all")

In [21]:
# tworzę "preview" poprzez stworzenie tabeli z nazw kolumn "df", zmieniam nazwę na "name", żeby zgadzała się z nazwą w "legend"
# po czym łączę to z "legend" w taki sposób, żeby pojawiły się tylko obecne 
preview = df.columns.to_frame().rename(columns={0:"name"}).merge(legend, on="name", how="left")
preview
# !!!!!!!!!!!!!!!! dodać do tego liczbę unikalnych wartości?????????????

,name,description
0,loan_amnt,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."
1,funded_amnt,The total amount committed to that loan at that point in time.
2,funded_amnt_inv,The total amount committed by investors for that loan at that point in time.
3,term,The number of payments on the loan. Values are in months and can be either 36 or 60.
4,int_rate,Interest Rate on the loan
5,installment,The monthly payment owed by the borrower if the loan originates.
6,grade,LC assigned loan grade
7,sub_grade,LC assigned loan subgrade
8,emp_title,The job title supplied by the Borrower when applying for the loan.*
9,emp_length,Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.


In [22]:
# kolejne: po usunięciu danych z przyszłości analiza konkretnych kolumn
# te z przyszłości: last_fico_range_high, last_fico_range_low, acc_now_delinq, delinq_amnt

Po usunięciu danych z przyszłości przychodzi czas na analizę kolumn i sprawdzenie, czy powinienem je jakoś zmodyfikować. Chcę sprawdzić kolumny z kilkoma wartościami i sprawdzić, czy mogę je jakoś zmodyfikować, oraz chcę usunąć znaki specjalne lub inne niepotrzebne informacje. Upewnię się również, czy dane w kolumnach są odpowiedniego typu.


In [23]:
# kolumna term opisuje, na jaki czas został przyznany kredyt i zwiera 2 wartości - 36 months i 60 months
print(df.term.unique())

[' 36 months' ' 60 months']


In [24]:
# zmieniam dane w kolumnie na dane liczbowe, bez dodatkowej informacji o tym, że są to miesiące, za to w nazwie kolumny dodaję "months"
df = df.rename(columns={"term": "term_months"})
df.term_months = df.term_months.map({" 36 months":36, " 60 months":60}).astype("int32")
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term_months,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,acc_now_delinq,delinq_amnt,pub_rec_bankruptcies,tax_liens,debt_settlement_flag
0,5000.0,5000.0,4975.0,36,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,735.0,739.0,1.0,3.0,0.0,13648.0,83.7%,9.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,Jan-2015,171.62,Aug-2018,739.0,735.0,0.0,0.0,0.0,0.0,N
1,2500.0,2500.0,2500.0,60,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,car,bike,309xx,GA,1.00,0.0,Apr-1999,740.0,744.0,5.0,3.0,0.0,1687.0,9.4%,4.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,Apr-2013,119.66,Oct-2016,499.0,0.0,0.0,0.0,0.0,0.0,N
2,2400.0,2400.0,2400.0,36,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,735.0,739.0,2.0,2.0,0.0,2956.0,98.5%,10.0,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,Jun-2014,649.91,Jun-2017,739.0,735.0,0.0,0.0,0.0,0.0,N
3,10000.0,10000.0,10000.0,36,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,other,personel,917xx,CA,20.00,0.0,Feb-1996,690.0,694.0,1.0,10.0,0.0,5598.0,21%,37.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,Jan-2015,357.48,Apr-2016,604.0,600.0,0.0,0.0,0.0,0.0,N
4,3000.0,3000.0,3000.0,60,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Fully Paid,other,Personal,972xx,OR,17.94,0.0,Jan-1996,695.0,699.0,0.0,15.0,0.0,27783.0,53.9%,38.0,4066.908161,4066.91,3000.00,1066.91,0.00,0.0,0.00,Jan-2017,67.30,Apr-2018,684.0,680.0,0.0,0.0,0.0,0.0,N


In [25]:
# usuwam "%" z kolumny "int_rate" i zmieniam typ na float
df.int_rate = df.int_rate.str.replace("%", "").astype(float)
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term_months,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,acc_now_delinq,delinq_amnt,pub_rec_bankruptcies,tax_liens,debt_settlement_flag
0,5000.0,5000.0,4975.0,36,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,735.0,739.0,1.0,3.0,0.0,13648.0,83.7%,9.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,Jan-2015,171.62,Aug-2018,739.0,735.0,0.0,0.0,0.0,0.0,N
1,2500.0,2500.0,2500.0,60,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,car,bike,309xx,GA,1.00,0.0,Apr-1999,740.0,744.0,5.0,3.0,0.0,1687.0,9.4%,4.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,Apr-2013,119.66,Oct-2016,499.0,0.0,0.0,0.0,0.0,0.0,N
2,2400.0,2400.0,2400.0,36,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,735.0,739.0,2.0,2.0,0.0,2956.0,98.5%,10.0,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,Jun-2014,649.91,Jun-2017,739.0,735.0,0.0,0.0,0.0,0.0,N
3,10000.0,10000.0,10000.0,36,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,other,personel,917xx,CA,20.00,0.0,Feb-1996,690.0,694.0,1.0,10.0,0.0,5598.0,21%,37.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,Jan-2015,357.48,Apr-2016,604.0,600.0,0.0,0.0,0.0,0.0,N
4,3000.0,3000.0,3000.0,60,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Fully Paid,other,Personal,972xx,OR,17.94,0.0,Jan-1996,695.0,699.0,0.0,15.0,0.0,27783.0,53.9%,38.0,4066.908161,4066.91,3000.00,1066.91,0.00,0.0,0.00,Jan-2017,67.30,Apr-2018,684.0,680.0,0.0,0.0,0.0,0.0,N


In [26]:
# co zrobic z  grade i sub_grade? czy wystarczy zachować tylko jedną z tych kolumn? czy może jakoś zamienić? :(

In [27]:
# emp_title - do zbadania
# df.emp_title.unique().tolist()
print("nunique:", df.emp_title.nunique())
print("nan:", df.emp_title.isna().sum()) # NaN zamienić na None
print("duplicated:", df.emp_title.duplicated().sum())

nunique: 30658
nan: 2626
duplicated: 11876


In [28]:
# emp_length
print("nunique:", df.emp_length.nunique())
print("nan:", df.emp_length.isna().sum())
print("duplicated:", df.emp_length.duplicated().sum())
print("unique:", df.emp_length.unique())
df.emp_length = df.emp_length.map({"< 1 year":0, "1 year":1, "2 years":2, "3 years":3, "4 years":4, "5 years":5, "6 years":6,
                  "7 years":7, "8 years":8, "9 years":9, "10+ years":10, "nan":0})
df.head()

nunique: 11
nan: 1112
duplicated: 42523
unique: ['10+ years' '< 1 year' '1 year' '3 years' '8 years' '9 years' '4 years'
 '5 years' '6 years' '2 years' '7 years' nan]


,loan_amnt,funded_amnt,funded_amnt_inv,term_months,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,acc_now_delinq,delinq_amnt,pub_rec_bankruptcies,tax_liens,debt_settlement_flag
0,5000.0,5000.0,4975.0,36,10.65,162.87,B,B2,NaN,10.0,RENT,24000.0,Verified,Dec-2011,Fully Paid,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,735.0,739.0,1.0,3.0,0.0,13648.0,83.7%,9.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,Jan-2015,171.62,Aug-2018,739.0,735.0,0.0,0.0,0.0,0.0,N
1,2500.0,2500.0,2500.0,60,15.27,59.83,C,C4,Ryder,0.0,RENT,30000.0,Source Verified,Dec-2011,Charged Off,car,bike,309xx,GA,1.00,0.0,Apr-1999,740.0,744.0,5.0,3.0,0.0,1687.0,9.4%,4.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,Apr-2013,119.66,Oct-2016,499.0,0.0,0.0,0.0,0.0,0.0,N
2,2400.0,2400.0,2400.0,36,15.96,84.33,C,C5,NaN,10.0,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,735.0,739.0,2.0,2.0,0.0,2956.0,98.5%,10.0,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,Jun-2014,649.91,Jun-2017,739.0,735.0,0.0,0.0,0.0,0.0,N
3,10000.0,10000.0,10000.0,36,13.49,339.31,C,C1,AIR RESOURCES BOARD,10.0,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,other,personel,917xx,CA,20.00,0.0,Feb-1996,690.0,694.0,1.0,10.0,0.0,5598.0,21%,37.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,Jan-2015,357.48,Apr-2016,604.0,600.0,0.0,0.0,0.0,0.0,N
4,3000.0,3000.0,3000.0,60,12.69,67.79,B,B5,University Medical Group,1.0,RENT,80000.0,Source Verified,Dec-2011,Fully Paid,other,Personal,972xx,OR,17.94,0.0,Jan-1996,695.0,699.0,0.0,15.0,0.0,27783.0,53.9%,38.0,4066.908161,4066.91,3000.00,1066.91,0.00,0.0,0.00,Jan-2017,67.30,Apr-2018,684.0,680.0,0.0,0.0,0.0,0.0,N


In [29]:
# home_ownership
print("nunique:", df.home_ownership.nunique())
print("nan:", df.home_ownership.isna().sum())
print("duplicated:", df.home_ownership.duplicated().sum())

nunique: 5
nan: 0
duplicated: 42530


In [30]:
# verification_status



In [31]:
# issue_d

In [32]:
# loan_status
# Y

In [33]:
# purpose + title (bardzo podobne wartości)

In [34]:
# zip_code addr_state ----- czy to jest potrzebne? (bardzo dużo kategorii)

In [35]:
# dti

In [36]:
# delinq_2yrs

In [377]:
# sprawdzam typ danych w każdej kolumnie
df.dtypes

loan_amnt                  float64
funded_amnt                float64
funded_amnt_inv            float64
term_months                  int32
int_rate                   float64
installment                float64
grade                       object
sub_grade                   object
emp_title                   object
emp_length                  object
home_ownership              object
annual_inc                 float64
verification_status         object
issue_d                     object
loan_status                 object
purpose                     object
title                       object
zip_code                    object
addr_state                  object
dti                        float64
delinq_2yrs                float64
earliest_cr_line            object
fico_range_low             float64
fico_range_high            float64
inq_last_6mths             float64
open_acc                   float64
pub_rec                    float64
revol_bal                  float64
revol_util          